<a href="https://colab.research.google.com/github/ArpitKadam/ColabNotebooks/blob/main/Source_Code_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain chromadb tiktoken
!pip install -q langchain_core langchain_community
!pip install -q langchain-chroma langchain-text-splitters
!pip install -q unstructured
!pip install -q esprima pdfminer
!pip install -q langchain_groq
!pip install -q GitPython
!pip install langchain_ollama ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.0 MB/s

In [ ]:
from IPython.display import display, Markdown
from git import Repo
from langchain_text_splitters import Language
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import TextLoader, UnstructuredImageLoader, UnstructuredHTMLLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

In [ ]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
!mkdir test_repo

In [ ]:
repo_path = "/content/test_repo"
repo = Repo.clone_from("https://github.com/ArpitKadam/DocuXment-App", to_path=repo_path)

In [ ]:
python_loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=100)
)

In [ ]:
text_loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*.txt"
)

In [ ]:
for path in Path(repo_path).rglob("*.md"):
    markdown_loader = UnstructuredMarkdownLoader(path)

In [ ]:
for path in Path(repo_path).rglob("*.log"):
    log_loader = TextLoader(path)

In [ ]:
for path in Path(repo_path).rglob("*.html"):
  html_loader = UnstructuredHTMLLoader(path, mode="elements")

In [ ]:
js_loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".js"],
    parser=LanguageParser(language=Language.JS, parser_threshold=100)
)

In [ ]:
documents = python_loader.load() + text_loader.load() + markdown_loader.load() + log_loader.load() + html_loader.load() + js_loader.load()

In [ ]:
len(documents)

66

In [ ]:
Markdown(documents[6].page_content)

def classify_question_complexity(question):
    """Enhanced question complexity classification for competition optimization"""
    question_lower = question.lower()
    
    # High complexity indicators
    high_complexity_indicators = [
        "compare", "analyze", "relationship", "interaction", "simultaneously", 
        "while", "also request", "also ask", "both", "multiple", "various", "different",
        "scenario", "interaction", "contrast", "evaluate"
    ]
    
    # Medium complexity indicators  
    medium_complexity_indicators = [
        "covered", "eligible", "benefit", "claim", "waiting period", "exclusion",
        "limitation", "condition", "procedure", "process", "requirement"
    ]
    
    # Simple question indicators
    simple_indicators = ["what is", "define", "meaning", "who is", "when", "where"]
    
    # Calculate complexity score
    complexity_score = 0
    complexity_score += sum(1 for indicator in high_complexity_indicators if indicator in question_lower)
    complexity_score += len([char for char in question if char == "?"]) - 1  # Multiple questions
    complexity_score += question.count(",") // 2  # Multiple clauses
    complexity_score += question.count(" and ") + question.count(" or ")  # Logical operators
    
    # Length-based complexity
    if len(question.split()) > 25:
        complexity_score += 2
    elif len(question.split()) > 15:
        complexity_score += 1
    
    # Determine final complexity
    if complexity_score >= 4:
        return "complex"
    elif complexity_score >= 2 or any(indicator in question_lower for indicator in medium_complexity_indicators):
        return "medium"
    elif any(indicator in question_lower for indicator in simple_indicators):
        return "simple"
    else:
        return "medium"

In [ ]:
for d in documents[:5]:
    print(d.metadata)


{'source': '/content/test_repo/api.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}
{'source': '/content/test_repo/api.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}
{'source': '/content/test_repo/api.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}
{'source': '/content/test_repo/api.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}
{'source': '/content/test_repo/api.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}


In [ ]:
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    Language,
    RecursiveCharacterTextSplitter
)

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=1000,
    chunk_overlap=200
)

js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS,
    chunk_size=600,
    chunk_overlap=100
)

markdown_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n## ", "\n### ", "\n\n", "\n", " "]
)

log_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    separators=["\n", " "]
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

In [ ]:
split_docs = []

for doc in documents:
  ext = doc.metadata['source'].split(".")[-1].lower()

  if ext == "py":
    split_docs.extend(python_splitter.split_documents([doc]))

  elif ext == "js":
    split_docs.extend(js_splitter.split_documents([doc]))

  elif ext in [".md", ".html", ".htm"]:
    split_docs.extend(markdown_splitter.split_documents([doc]))

  elif ext == ".log":
    split_docs.extend(log_splitter.split_documents([doc]))

  else:
    split_docs.extend(text_splitter.split_documents([doc]))

In [ ]:
print("Before:", len(documents))
print("After:", len(split_docs))

Before: 66
After: 135


In [ ]:
clean_docs = filter_complex_metadata(split_docs)

In [ ]:
print("Before:", len(documents))
print("After:", len(split_docs))
print("After Cleaning:", len(clean_docs))

Before: 66
After: 135
After Cleaning: 135


In [ ]:
embeddings = OllamaEmbeddings(
  model="embeddinggemma"
)

llm = ChatGroq(
    model="openai/gpt-oss-20b"
)

In [ ]:
vector_db = Chroma.from_documents(
    documents=clean_docs,
    embedding=embeddings,
    persist_directory="db"
)

In [ ]:
retriever = vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5}
)

In [ ]:
retriever.invoke("HTTPBearer")

[Document(id='6a176ee2-c469-4283-ad82-cefc15abfe20', metadata={'element_id': '54ed4443a2b2b983bd1b5e402e7151ad', 'source': '/content/test_repo/templates/index.html', 'filename': 'index.html', 'parent_id': '4407651d967591e0ea5895ca6025b300', 'filetype': 'text/html', 'category_depth': 2, 'last_modified': '2025-12-16T10:52:53', 'file_directory': '/content/test_repo/templates', 'category': 'Title'}, page_content='Sources'),
 Document(id='2e56fed5-3d67-45f3-919d-05939a84b675', metadata={'parent_id': '7945d65501b4b04706b5679364dadc34', 'filename': 'index.html', 'category': 'UncategorizedText', 'filetype': 'text/html', 'source': '/content/test_repo/templates/index.html', 'element_id': 'e9b77d8e7252d0fe1453851350cac46e', 'file_directory': '/content/test_repo/templates', 'last_modified': '2025-12-16T10:52:53'}, page_content='{{ message }}'),
 Document(id='e49f4f40-e85b-44c5-8ec4-68259bf9c593', metadata={'category_depth': 1, 'parent_id': '6a498e027f0d909e8574ef91fbe5d6ac', 'file_directory': '/co

In [ ]:
prompt = ChatPromptTemplate.from_messages([
      (
        "system",
        """
        You are an expert Source Code Analyzer and Software Architect.

        Your role:
        - Analyze source code, configuration files, documentation, and logs.
        - Explain functionality, architecture, control flow, and dependencies.
        - Identify key files, functions, classes, and modules.
        - Reason strictly from the provided context (retrieved code chunks).
        - If information is missing, say so explicitly — do NOT hallucinate.

        Guidelines:
        - Prefer precise, technical explanations.
        - Reference filenames, paths, or functions when available.
        - For “how it works” questions, explain step-by-step execution flow.
        - For “why” questions, infer intent only if supported by code or docs.
        - For debugging questions, point to exact lines or modules responsible.
        - For summaries, group by components or layers (API, service, utils, config).
        - Keep answers concise but technically complete.
        """
      ),

      # Conversational memory
      MessagesPlaceholder(variable_name="history"),

      (
        "human",
        """
        Retrieved Context (source code & docs):
        {context}

        User Question:
        {input}
        """
      )
])


In [ ]:
rag_chain = (
    {
        # Extract ONLY the user query for retrieval
        "context": RunnableLambda(lambda x: x["input"]) | retriever,

        # Pass user input to the prompt
        "input": RunnableLambda(lambda x: x["input"]),

        # 🔑 PASS HISTORY THROUGH
        "history": RunnableLambda(lambda x: x["history"]),
    }
    | prompt
    | llm
)


In [ ]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [ ]:
rag_chain_with_memory = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
config={"configurable": {"session_id": "user-2"}}

In [ ]:
response = rag_chain_with_memory.invoke(
    {"input": "What is the purpose of function build_faiss_for_urls"},
    config=config
)

Markdown(response.content)

**Purpose of `build_faiss_for_urls`**

The `build_faiss_for_urls` function is a helper that takes a list of URLs, downloads or fetches the content at those URLs, and then creates a FAISS (Facebook AI Similarity Search) vector‑store index from that content.  

In short, it turns arbitrary web resources into a searchable vector database that can later be queried with embeddings.

**High‑level workflow**

1. **Iterate over the supplied URLs**  
   ```python
   for link in urls:
   ```

2. **Determine the MIME type (optional)**  
   It sends a `HEAD` request to the URL to inspect the `Content‑Type` header.  
   ```python
   head = requests.head(link, allow_redirects=True, timeout=10)
   ctype = (head.headers.get("Content-Type") or "").lower()
   ```
   This can be used to decide how to load the file (e.g., PDF, text, image).

3. **Identify file extension**  
   ```python
   ext = Path(urlparse(link).path).suffix.lower()
   ```
   The extension (`.pdf`, `.txt`, `.jpg`, etc.) is used to pick an appropriate loader.

4. **Download or load the content**  
   Depending on the MIME type and extension, the function would:
   * Use `PyPDFLoader` for PDFs,
   * Use `requests` or `urllib` to fetch HTML/text,
   * Use `pytesseract` for images (OCR),
   * Or any other loader available in the project.

5. **Create a `Document` object**  
   The fetched content is wrapped in a `langchain_core.documents.Document` instance, which includes metadata such as the URL, file type, and raw text.

6. **Split the text into chunks**  
   A `RecursiveCharacterTextSplitter` (imported in `api.py`) would break the raw text into smaller, manageable pieces suitable for embeddings.

7. **Generate embeddings**  
   Using `NVIDIAEmbeddings` (or another embedding provider), each chunk is converted into a dense vector.

8. **Store vectors in FAISS**  
   The vectors are added to a `langchain_community.vectorstores.FAISS` instance, producing an in‑memory similarity search index.

9. **Return or expose the FAISS index**  
   The function finally returns the FAISS object (or stores it globally) so that downstream API endpoints can query it.

**Why this is useful**

- **Unified search**: By turning disparate web resources into a single vector index, you can run semantic queries across all of them.
- **Scalability**: FAISS is highly optimized for fast nearest‑neighbor search on large embeddings collections.
- **Modularity**: The function separates the concerns of fetching data and building the index, making the rest of the codebase cleaner.

**Missing details**

The snippet provided stops after determining the file extension. The full implementation would include the steps above (loading, splitting, embedding, indexing). If you need the exact code for those steps, the rest of the `flask_app.py` file (or a helper module) would contain them.